In [2]:
import random

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pickle

import utils.features.signature as signature
import datasets
import utils

## LOADING THE DATA

In [3]:
signature_data = []
with open("./data/tdsc/tumors/signature.pkl", "rb") as sig_file:
    signature_data = pickle.load(sig_file)
    
for i in range(len(signature_data)):
    x, y = signature_data[i]
    num_zero_slices = 180 - len(x)
    zero_slices = [np.zeros(360, dtype=np.float32) for i in range(num_zero_slices)]
    x = x + zero_slices
    x=np.array(x, dtype=np.float32)
    y=np.array(y, dtype=np.float32)
    x = torch.tensor(x)
    y = torch.tensor(y).unsqueeze(0)
    signature_data[i] = (x,y)

In [4]:
class SequenceClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device='cpu'):
        super(SequenceClassifier, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size * 180, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)

        out = out.reshape(out.shape[0], -1)
        
        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [5]:
num_epochs = 100
learning_rate = 10e-4
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()

In [6]:
num_correct_predictions = 0
num_samples = 100

for idx in tqdm(range(len(signature_data))):
    train_data = signature_data[:idx] + signature_data[idx+1:]
    model = SequenceClassifier(360, 20, 1, 2, device=device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    test_sample = signature_data[idx]
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)
    for epoch in range(num_epochs):
        data_loop = train_loader
        for data in data_loop:
            
            x, y = data
            x = x.to(device).squeeze(1)
            y = y.to(device).squeeze(1).long()  
                    
            predictions = model(x)
            loss = criterion(predictions, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                
        total = 0
        num_corrects = 0
        
    for data in train_loader:
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
        
    print(f"Testing sample #{idx}: train acc is: {num_corrects}/{total}")
                        
    x, y = test_sample
    x = x.to(device).unsqueeze(0)
    y = y.to(device).unsqueeze(0).long()  

    model.eval()

    with torch.no_grad():

        predictions = model(x)
        _, predictions = predictions.max(1)
        num_correct_predictions += (predictions == y).float().sum()
        
    model.train()  
        
print(f"Finished: test acc is: {num_correct_predictions}/{num_samples}")

  0%|          | 0/100 [00:00<?, ?it/s]

Testing sample #0: train acc is: 83.0/99
Testing sample #1: train acc is: 92.0/99
Testing sample #2: train acc is: 85.0/99
Testing sample #3: train acc is: 83.0/99
Testing sample #4: train acc is: 96.0/99
Testing sample #5: train acc is: 89.0/99
Testing sample #6: train acc is: 83.0/99
Testing sample #7: train acc is: 96.0/99
Testing sample #8: train acc is: 86.0/99
Testing sample #9: train acc is: 86.0/99
Testing sample #10: train acc is: 81.0/99
Testing sample #11: train acc is: 85.0/99
Testing sample #12: train acc is: 84.0/99
Testing sample #13: train acc is: 85.0/99
Testing sample #14: train acc is: 83.0/99
Testing sample #15: train acc is: 84.0/99
Testing sample #16: train acc is: 83.0/99
Testing sample #17: train acc is: 89.0/99
Testing sample #18: train acc is: 86.0/99
Testing sample #19: train acc is: 96.0/99
Testing sample #20: train acc is: 87.0/99
Testing sample #21: train acc is: 89.0/99
Testing sample #22: train acc is: 89.0/99
Testing sample #23: train acc is: 91.0/99
Te

In [8]:
print(f"Finished: test acc is: {num_correct_predictions}/{num_samples}%")

Finished: test acc is: 50.0/100%
